In [0]:
DATA_DIR = '/datascope/subaru/data/catalogs/xshooter/'
LIST_FILE = DATA_DIR + '_list.csv'

PROJECT_PATH = '/home/dobos/project/ga_isochrones/python:' + \
    '/home/dobos/project/ga_pfsspec_all/python:' + \
    '/home/dobos/project/pysynphot'

GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/phoenix/phoenix_HiRes'

# XShooter DR3

Verro et al.
* Paper in A&A: https://www.aanda.org/articles/aa/pdf/2022/04/aa42388-21.pdf
* Data in Vizier: http://cdsarc.u-strasbg.fr/viz-bin/cat/J/A+A/660/A34

This moderate-to-high resolution, near-ultravioletto-near-infrared (350–2480 nm, R ∼ 10 000) spectral library is composed of 830 stellar spectra of 683 stars. 

CSV file donwloaded from Vizier using the SQL search tool with the query:

```sql
SELECT [default] FROM obscore
WHERE obs_collection='J/A+A/660/A34'
```

In [0]:
import os, sys
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import notebook as tqdm
from astropy.io import fits
from glob import glob

In [0]:
# Allow load project as module
for p in reversed(PROJECT_PATH.split(':')):
    sys.path.insert(0, p)

os.environ['PYTHONPATH'] = PROJECT_PATH.split(':')[0]

In [0]:
os.environ['PYTHONPATH']

In [0]:
%matplotlib inline

# Load xshooter spectrum

In [0]:
df = pd.read_csv(LIST_FILE)
df.shape

In [0]:
df.columns

In [0]:
len(df['obs_id'].unique())

# Download data

In [0]:
t = tqdm.tqdm(total=df.shape[0])
for i, r in df.iterrows():
    url = r['access_url']
    fn = os.path.join(DATA_DIR, r['obs_id'])
    if not os.path.isfile(fn):
        res = requests.get(url)
        with open(fn, 'wb') as f:
            f.write(res.content)
    t.update(1)

# Read all FITS headers and write to file

In [0]:
fn = os.path.join(DATA_DIR, '_headers.csv')
if not os.path.isfile(fn):
    files = glob(os.path.join(DATA_DIR, '*.fits'))
    df = None

    for fn in files:
        hdus = fits.open(fn, memmap=False)
        headers = { k: hdus[0].header[k] for k in hdus[0].header.keys() }
        
        if df is None:
            df = pd.DataFrame(headers, index=['XSL_ID'])
        else:
            df = df.append(headers, ignore_index=True)

df.shape

In [0]:
fn = os.path.join(DATA_DIR, '_headers.csv')
if not os.path.isfile(fn):
    df.to_csv(fn)

In [0]:
df

In [0]:
list(df['target_name'])

# Read FITS and plot

In [0]:
fn = os.path.join(DATA_DIR, 'xsl_spectrum_MD004_merged_scl.fits')
hdus = fits.open(fn, memmap=False)
for h in hdus:
    print(h)

In [0]:
hdus[0].header

In [0]:
hdus[0].header['XSL_ID'], hdus[0].header['OBJECT'], hdus[0].header['SNR'], hdus[0].header['SPEC_RES']


# BARY_COR      / Barycentric radial velocity correction value 
# REST_VIS      / VIS cz value in km/s 
# REST_UVB      / UVB cz value in km/s 
# AV_VAL        / Total extinction in V

HD014938 in Simbad: https://simbad.u-strasbg.fr/simbad/sim-id?protocol=html&Ident=HD014938

M/H

In [0]:
wave = hdus[1].data['WAVE']
flux = hdus[1].data['FLUX']
flux_dr = hdus[1].data['FLUX_DR'] if 'FLUX_DR' in hdus[1].data else None      # dereddened flux?
flux_err = hdus[1].data['ERR']

In [0]:
flux

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 3), dpi=240)

#mask = (790 <= wave) & (wave <= 796)
mask = np.full_like(wave, True, dtype=np.bool)
print(mask.sum())

if flux_dr is not None:
    ax.plot(wave[mask], flux_dr[mask], lw=0.3)
ax.plot(wave[mask], flux[mask], lw=0.3)
#ax.plot(wave[mask], flux_err[mask], lw=0.3)

ax.grid()

# Spectral resolution

In [0]:
wref = (0.5 * (wave[1:] + wave[:-1]))

mask = (700 <= wref) & (wref <= 900)

R = wref / np.diff(wave)
R[mask].mean(), R[mask].std()

# Load model grid

In [0]:
from pfs.ga.pfsspec.core.grid import ArrayGrid
from pfs.ga.pfsspec.stellar.grid import ModelGrid
from pfs.ga.pfsspec.stellar.grid.bosz import Bosz
from pfs.ga.pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Phoenix(), ArrayGrid)
grid.preload_arrays = False
grid.load(fn, format='h5')